# OpenMC Post-Processing

### This notebook reads OpenMC output files for all the reactors, and computes SNF metrics, including mass, volume, activity, radiotoxicity and decay heat

In [ ]:
# Import the python modules and openmc modules 
%matplotlib inline
import openmc
import openmc.deplete
import math
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Path to the decay chain file used for the simulation
openmc.config['chain_file']='./nuclear_data/chain_endfb71_pwr.xml'

## Functions

In [3]:
def get_results(reactor, primary_path):
    """
    Read and return the results from the depletion simulation for one reactor
    """
    ## ---- Parameters: 
    # reactor (str) - reactor name
    # primary_path (str) - a path to the directory in which openmc files are stored
    ## ---- Returns: 
    # results - openmc depletion calculation results
    
    results_file = primary_path+ "depletion_results.h5" # path to the depletion_results.h5 file
    results      = openmc.deplete.Results(results_file)
    
    return results

In [4]:
def get_crit_param(results):
    """
    Read the result file and create a list with [time (d), keff, k uncerntainty, discharge step] 

    """
    ## ---- Parameters: 
    #  results - openmc depletion calculation results
    ## ---- Returns: 
    #  crit_param (tuple) - the criticality prameters including:
    #    - time - time in years
    #    - keff - k-effective as a function of time
    #    - discharge - discharge time step
    
    time, k     = results.get_keff()
    keff        = k[:,0]
    n_discharge = np.max(np.nonzero(keff))
    crit_param  = (time/(24*60*60), keff, n_discharge)
    
    return crit_param

In [5]:
def get_keff_final(crit_param):
    """
    Return the last value of keff
    """
    ## ---- Parameters: 
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  keff_final including
    #  time[n] - the time of discharge
    #  keff[n] - k_eff at the time of discharge
    
    n_discharge = crit_param[2]
    time        = crit_param[0]
    keff        = crit_param[1]

    keff_final  = (time[n_discharge],keff[n_discharge])
    
    return keff_final

In [6]:
def number_steps(crit_param):
    """
    Return the number of steps of the enire simulation
    """
    ## ---- Parameters: 
    #  crit_param (tuple) - the result from get_crit_param(results)
    #  reactor(str) - reactor name
    ## ---- Returns: 
    #  n - the number of time steps
    
    time  = crit_param[0]
    n_tot = len(time)
    
    return n_tot

In [7]:
def conv_GWe_y(value, reactor): # value should be given in value/gHM
    """
    Convert the output data (activity, decay heat...) from /gHM to /GWe.y
    """
    ## ---- Parameters: 
    #  value - the value to be converted to per electricity production
    #  reactor(str) - reactor name
    ## ---- Returns: 
    #  value per electricity production
    
    param = get_parameters(reactor)
    
    return value*365/(param[2]*param[1]*(10**-6))

In [8]:
def get_snf_composition(primary_path, results, crit_param, id_mat):
    """
    Radionucide Composition of SNF---
    Create a collection of OpenMC Material with the spent fuel material at each stage
    crit_param is a list containing [time, keff, kun, discharge step]
    id_mat correspond to the ID of the depleted material (usually '1' for the fuel)
    """
    ## ---- Parameters: 
    #  primary_path (str) - a path to the directory in which openmc files are stored
    #  results - openmc depletion calculation results
    #  crit_param (tuple) - the result from get_crit_param(results)
    #  id_mat - the ID of the depleted material (usually '1' for the fuel)
    ## ---- Returns: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    
    n_tot           = number_steps(crit_param)
    snf_composition = [results.export_to_materials(burnup_index=i,path=primary_path+ 'materials.xml')[int(id_mat)-1] for i in range(n_tot) ]# burnup index= step number, path=path to the materials.xml file use for the simulation

    return snf_composition

In [3]:
def get_power(param, crit_param):
    """
    Burnup at each time steps---
    Return a list with the value of the input power during operation and discharge
    """
    ## ---- Parameters: 
    #  param - reactor parameters
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  power - total power out during operation and after discharge
    
    n_discharge = crit_param[2] # discharge step
    n_tot       = number_steps(crit_param) # total time step
    
    power   = [param[0] for i in range(n_discharge+1)]
    cooling = [0 for i in range(n_tot - n_discharge-1)]
    power   = power+cooling
    
    return power

In [10]:
def get_burnup(power, snf_composition, crit_param): # time must be given in days
    """
    Return a list with the value of the burnup at each time step
    """
    ## ---- Parameters: 
    #  power - total power out during operation and after discharge from power_evolution()  
    #  crit_param (tuple) - the result from get_crit_param(results)
    #  snf_composition - a collection of OpenMC Material with the spent nuclear fuel material composition at each time step
    ## ---- Returns: 
    #  burnup_step - burnup at each time step
       
    fis_mass = snf_composition[0].fissionable_mass  
    time     = crit_param[0]
    
    burnup =[time[i]*power[i]/((10**3)*fis_mass) for i in range(len(time))]
    
    return burnup

In [11]:
def get_activity(snf_composition, crit_param):
    """
    Activity of the spent fuel---
    Create a dataframe with the activity of the spent fuel at each time step, for each nuclide
    material collec is the collection of materials obtained with material_collection
    """
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  crit_param (tuple) - the result from crit_param(results)
    ## ---- Returns: 
    #  df_snf - dataframe containing the activity of each radionuclide in SNF at each time step
    
    n_tot = number_steps(crit_param)    
    time  = crit_param[0]
    
    df_act = pd.DataFrame()
    for i in range (n_tot):
        act       = snf_composition[i].get_activity(by_nuclide=True, units='Bq/g')
        df_act[i] = pd.DataFrame.from_dict(act, orient='index', columns=[i])
    
    df_act.columns=time
    
    return df_act

In [12]:
def get_activity_tot(snf_composition, crit_param): 
    """
    Create a dataframe with the total activity(Bq/g) of the spent fuel at each time step
    """
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  df_act - dataframe containing the total activity(Bq/g) of the spent fuel at each time step

    n_tot = number_steps(crit_param)    
    time  = crit_param[0]
    
    act_tot = [snf_composition[i].get_activity(by_nuclide=False, units='Bq/g') for i in range(n_tot)]
    df_act  = pd.DataFrame(act_tot, index=time, columns=['Total_Activity (Bq/g)'])
    
    return df_act

In [13]:
def get_decay_heat(snf_composition, crit_param):
    """
    Decay heat of the spent fuel--
    Create a dataframe with the decay heat of the spent fuel fro at each time step
    """
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  df_DH - dataframe containing the decay heat (W/g) of the spent fuel at each time step

    n_tot = number_steps(crit_param)    
    time  = crit_param[0]
    
    df_DH = pd.DataFrame()
    for i in range(n_tot):
        DH       = snf_composition[i].get_decay_heat(units= 'W/g', by_nuclide=True)
        df_DH[i] = pd.DataFrame.from_dict(DH, orient='index', columns=[i])
    
    df_DH.columns=time
    
    return df_DH

In [14]:
def get_decay_heat_tot(snf_composition, crit_param):
    """
    Create a dataframe with the total decay heat of the spent fuel at each time step
    """
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  df_DH - dataframe containing the total decay heat (W/g) of the spent fuel at each time step
    n_tot = number_steps(crit_param)    
    time  = crit_param[0]
    
    DH_tot     = [snf_composition[i].get_decay_heat(units='W/g',by_nuclide=False) for i in range(n_tot)]
    df_DH_tot  = pd.DataFrame(DH_tot, index=time, columns=['Total_Decay_Heat (W/g)'])
    
    return df_DH_tot

In [15]:
def get_toxicity(snf_composition, crit_param):
    '''
    Radiotoxicity---
    Return the radiotoxicity for each nuclide
    '''
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  df_tox - dataframe containing the radiotoxicity for each nuclide at each time step
    
    # Path to the CSV file containing the DCF/ MPC factors 
    df_DCF = pd.read_csv('DCF_ingestion.csv', sep=';')
    df_DCF.set_index('Isotope', inplace=True) # Custom the dataframe
    df_DCF = df_DCF.iloc[1:,:].dropna()

    df_tox_mat = get_activity(snf_composition, crit_param) # dataframe containing the activity of each nuclide
    df_tox = df_tox_mat.merge(df_DCF, left_index=True, right_index=True, how='inner') # merge the 2 dataframe
    df_tox = df_tox.mul(df_tox['Conversion factors (Sv/Bq)'], axis='index') # multiply the activity by the dose conversion factor for each nuclide
    df_tox = df_tox.iloc[:, :-1]
    
    return df_tox

In [16]:
def get_toxicity_tot(snf_composition, crit_param):
    '''
    Return the total radiotoxicity for each time step
    '''
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  crit_param (tuple) - the result from get_crit_param(results)
    ## ---- Returns: 
    #  df_tox_tot - dataframe containing the total radiotoxicity at each time step
       
    time  = crit_param[0]
    
    df_tox = get_toxicity(snf_composition, crit_param)
    df_tox_tot = df_tox.sum(axis=0)
    df_tox_tot = df_tox_tot.transpose()
    df_tox_tot = df_tox_tot.set_axis(time)
    
    return df_tox_tot

In [17]:
def get_SNF_mass(param): 
    '''
    Return SNF mass kg/MWe.year
    '''
    ## ---- Parameters: 
    #  param - reactor parameters 
    ## ---- Returns: 
    #  snf_mass - dataframe containing the total decay heat (W/g) of the spent fuel at each time step
    
    burnup_discharge = param[2]
    efficiency       = param[1]
    snf_mass         = 365/(burnup_discharge*efficiency)
    
    return snf_mass

In [18]:
def get_SNF_volume(param, snf_composition):
    '''
    Return the SNF volume in m3/ GWe.year
    '''
    ## ---- Parameters: 
    #  snf_composition - a collection of OpenMC Material with the spent fuel material at each time step
    #  param - reactor parameters 
    ## ---- Returns: 
    #  snf_volume = dataframe containing the total decay heat (W/g) of the spent fuel at each time step   
    
    volume_model = param[3]
    
    fiss_mass  = snf_composition[0].fissionable_mass

    snf_mass   = get_SNF_mass(param)
    snf_volume = volume_model*snf_mass/fiss_mass
       
    return snf_volume

In [19]:
def get_I129(results):
    '''
    Calculate I129 concentration in atom/cm3
    '''
    ## ---- Parameters: 
    #  results - openmc depletion calculation results
    ## ---- Returns: 
    #  I129 - I129 concentration in atom/cm3
    
    _, I129 = results.get_atoms("1", "I129", nuc_units='atom/cm3', time_units='d')
    
    return I129

## Dataframe with all the results

In [20]:
def global_df(reactor, results):
    '''
    Return a dataframe containing the main metrics and parameters useful in the study
    '''
    df = pd.DataFrame()
    
    # Previous results
    param      = get_parameters(reactor)
    crit_param = get_crit_param(results)
    
    snf_composition = get_snf_composition(primary_path, results, crit_param, "1")
    
    n     = number_steps(crit_param)
    power = get_power(param, crit_param)
    
    # Gather the information for the dataframe 
    df['keff'] = crit_param[1]
    df.set_index(crit_param[0], inplace=True)
    df['burnup'] = get_burnup(power, snf_composition, crit_param)
    df['Power (W/cm)'] = power
    df['Activity (Bq/g)'] = get_activity_tot(snf_composition, crit_param)
    df['Activity (Ci/GWe.y)'] = conv_GWe_y(df['Activity (Bq/g)'], reactor)*27*(10**-12)
    df['Decay heat (W/g)']=get_decay_heat_tot(snf_composition, crit_param)
    df['Decay heat (W/GWe.y)'] = conv_GWe_y(df['Decay heat (W/g)'], reactor)
    df['Radiotoxicity (Sv/g)'] = get_toxicity_tot(snf_composition, crit_param)
    df['Radiotoxicity (Sv/GWe.y)'] = conv_GWe_y(df['Radiotoxicity (Sv/g)'], reactor)
    df['I129 (atom/cm3)'] = get_I129(results)
    df['I129 (atom/GWe.y)'] = df['I129 (atom/cm3)']*snf_composition[0].volume*365/(param[0]*get_keff_final(crit_param)[0]*param[1])*(10**9)
    df['Efficiency']=param[1]
    df['Volume Model (cm3)'] = param[3]
    df['SNF mass (t/GWe.y)'] = get_SNF_mass(param)
    df['SNF volume (m3/(GWe.y))'] = df['SNF mass (t/GWe.y)']*param[3]
    df['discharge step'] = crit_param[2]
    df['discharge time'] = get_keff_final(crit_param)[0]
    df['volume_Ueq']= snf_composition[0].volume
    df['mass U'] = snf_composition[0].fissionable_mass
    return df

In [21]:
def by_nuclide(reactor, results):
    '''
    Return an array of dataframes with the activity, radiotoxicity, decay heat per nuclide at each time step
    Make the comparison with detailed composition easier
    '''
    # Gather the parameters for the reactor
    param = get_parameters(reactor)
    crit_param = get_crit_param(results)
    snf_composition = get_snf_composition(primary_path, results, crit_param, "1")
    n_tot = number_steps(crit_param)
    time = crit_param[0] # time steps
    n_discharge = crit_param[2] # the time steps of the discharge step
    
    # Create the dataframe
    df_act = get_activity(snf_composition, crit_param) # activity
    df_DH  = get_decay_heat(snf_composition, crit_param) #decay heat
    df_tox = get_toxicity(snf_composition, crit_param) # radiotoxicity
    #Same dataframe in /GWe.y
    df_act_conv = conv_GWe_y(df_act, reactor)
    df_DH_conv  = conv_GWe_y(df_DH, reactor)
    df_tox_conv = conv_GWe_y(df_tox, reactor)
    return (df_act, df_DH, df_tox,time, n_discharge, df_act_conv, df_DH_conv, df_tox_conv)

In [22]:
def df_to_csv(reactor,primary_path):
    '''
    Create 2 csv files with the df_global dataframe
    '''
    results     = get_results(reactor, primary_path)
    df          = global_df(reactor, results) # dataframe containing the results for operation + discharge time 
    n_discharge = df['discharge step'][0]
    df_cooling  = df.iloc[n_discharge:].copy(deep=True) # dataframe containing the value only after discharge from the reactor
    df.to_csv(path_or_buf = primary_path+'out_'+reactor+'.csv', index=True, header=True)
    df_cooling.to_csv(path_or_buf = primary_path+'cooling'+reactor+'.csv', index=True, header=True)

In [23]:
def df_prop(df):
    '''
    Return a dataframe containing the contribution of each nuclide for the metric consider, contained in df
    '''
    total   = df.sum(axis=0)
    df_prop = df/total
    return df_prop

In [24]:
def by_nuclide_csv(reactor, primary_path):
    results = get_results(reactor, primary_path)
    # Dataframe with the waste management metrics by nuclide    
    df_act, df_DH, df_tox, time, tis_dis, df_act_conv, df_DH_conv, df_tox_conv=by_nuclide(reactor, results)
    
    # For each metrics, calculate the contribution of each metric
    act_prop = df_prop(df_act)
    DH_prop  = df_prop(df_DH)
    tox_prop = df_prop(df_tox)
        
    # Dataframe to export to csv for analysis    
    df_act.to_csv(path_or_buf=primary_path+'activity_'+reactor+'.csv', index=True, header=True)
    df_DH.to_csv(path_or_buf=primary_path+'decay_heat_'+reactor+'.csv', index=True, header=True)
    df_tox.to_csv(path_or_buf=primary_path+'radiotoxicity_'+reactor+'.csv', index=True, header=True)
    df_act_conv.to_csv(path_or_buf=primary_path+'activity_conv_'+reactor+'.csv', index=True, header=True)
    df_DH_conv.to_csv(path_or_buf=primary_path+'decay_heat_conv_'+reactor+'.csv', index=True, header=True)
    df_tox_conv.to_csv(path_or_buf=primary_path+'radiotoxicity_conv_'+reactor+'.csv', index=True, header=True)
    act_prop.to_csv(path_or_buf=primary_path+'activity_prop_'+reactor+'.csv', index=True, header=True)
    DH_prop.to_csv(path_or_buf=primary_path+'decay_heat_prop_'+reactor+'.csv', index=True, header=True)
    tox_prop.to_csv(path_or_buf=primary_path+'radiotoxicity_prop_'+reactor+'.csv', index=True, header=True)

## Example

## Single reactor

Set-up

In [48]:
# List of the reactors name to study 
reactor      = 'SFR'
primary_path = 'Results/'+reactor+'/'

# Necessary files
# rimary_path+ "depletion_results.h5" - openMC output file
# primary_path+ 'materials.xml' - openMC material file 
# DCF_ingestion.csv - the DCF/ MPC factors in the running directory 

def get_parameters(reactor): 
# Return the input parameters of the simulations [linear power (W/cm), thermal efficiency, discharge burnup, volume-to-mass ratio]
    if (reactor=='Ref_PWR'):
        parameters=[5.72*1000*1.26/30.48, 0.33, 50, 0.441]
    if (reactor=='SPWR'):
        parameters=[3.90*1000*1.26/30.48, 0.33, 15, 0.433]
    if (reactor=='HTGR'):
        parameters=[76969.02001294993, 0.40, 80, 21.8]
    if (reactor=='HPR'):
        parameters=[76969.02001294993, 0.33, 80, 21.8]
    if (reactor=='HTGR_FCM'):
        parameters=[190066.35554218251, 0.33, 80, 21.8]
    if (reactor=='SFR'):
        parameters=[250000, 0.4, 120, 0.912]
    return parameters

Run codes

In [49]:
# Create csv file containing the results and main parameters
df_to_csv(reactor, primary_path)

/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=6.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=8.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packa

In [50]:
# Create csv files detailed per nuclide for each reactor
by_nuclide_csv(reactor, primary_path)

/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=1.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=2.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=3.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=6.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Material instance already exists with id=7.
  warn(msg, IDWarning)
/Users/harukowainwright/anaconda3/envs/openmc-env/lib/python3.12/site-packa

## Multiple reactors 


In [ ]:
#reactors=['Ref_PWR', 'SPWR', 'HTGR', 'HPR', 'HTGR_FCM','SFR']

#for reactor in reactors:
#    primary_path='Results/'+reactor+'/'
#    df_to_csv(reactor, primary_path)

In [ ]:
#for reactor in reactors:
#    primary_path='Results/'+reactor+'/'
#    by_nuclide_csv(reactor,primary_path)